In [1]:
import os
import sys
import getpass
import fiona
import pandas as pd
import geopandas as gpd

user = getpass.getuser()

DVUTILS_LOCAL_CLONE_PATH = f"/Users/{user}/Documents/GitHub/dvutils"
sys.path.insert(0, DVUTILS_LOCAL_CLONE_PATH)
from utils_io import *

Info: Found credentials at: /Users/jcroff/Library/CloudStorage/Box-Box/dvutils-creds-jcroff.json


In [2]:
pba2050_path = f"/Users/{user}/Library/CloudStorage/Box-Box/Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/PBA50/Final Blueprint runs/Final Blueprint (s24)/BAUS v2.25 - FINAL VERSION"
pba2050_input_path = f"/Users/{user}/Library/CloudStorage/Box-Box/Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/PBA50/Current PBA50 Large General Input Data"
pba2050_geo_path = f"/Users/{user}/Library/CloudStorage/Box-Box/Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/BASIS/PBA50Plus/urbansim_geodata.gpkg"

## Read data

In [3]:
fiona.listlayers(pba2050_geo_path)

['urbansim_parcels', 'urbansim_parcels_topo_fix']

In [4]:
# read urbansim data
df15 = pd.read_csv(os.path.join(pba2050_path, "run182_parcel_data_2015.csv"))
df50 = pd.read_csv(os.path.join(pba2050_path, "run182_parcel_data_2050_UBI.csv"))
df_parcel = pd.read_csv(os.path.join(pba2050_input_path, "2020_08_17_parcel_to_taz1454sub.csv"))
gdf_parcel = gpd.read_file(pba2050_geo_path, layer="urbansim_parcels_topo_fix", driver="GPKG")

In [5]:
# update county based on manual_county
cnty_id_map = {
    1: "Alameda",
    97: "Sonoma",
    13: "Contra Costa",
    55: "Napa",
    75: "San Francisco",
    81: "San Mateo",
    41: "Marin",
    85: "Santa Clara",
    95: "Solano",
}
df_parcel["urbansim_county"] = df_parcel.manual_county.map(cnty_id_map)

## Join data

In [6]:
df_parcel.rename(columns={"PARCEL_ID": "parcel_id"}, inplace=True)

In [7]:
# join plan bay area data 
out_cols = ["parcel_id", "residential_units", "deed_restricted_units"]
df_pba2050 = pd.merge(df15[out_cols], df50[out_cols], on="parcel_id", suffixes=("_2015", "_2050"))
df_pba2050 = pd.merge(df_pba2050, df_parcel[["parcel_id", "urbansim_county"]], on="parcel_id")

In [8]:
# join geodata
gdf_pba2050 = pd.merge(gdf_parcel[["parcel_id", "geometry"]], df_pba2050, on="parcel_id")

In [9]:
reorder_cols = [
    "parcel_id",
    "urbansim_county",
    "residential_units_2015",
    "deed_restricted_units_2015",
    "residential_units_2050",
    "deed_restricted_units_2050",
    "geometry",
]
gdf_pba2050 = gdf_pba2050[reorder_cols]

In [10]:
# save to geojson file
out_path = f"/Users/{user}/Library/CloudStorage/Box-Box/DataViz Projects/Spatial Analysis and Mapping/Revenue Measure 2024/Data"
gdf_pba2050.to_file(os.path.join(out_path, "pba2050_residential_units.geojson"), driver="GeoJSON")